# Tutorial 2 - Read, manipulate and analyze the 2A PMW products

First, let's import the package required in this tutorial.

In [45]:
import gpm_api
import datetime

Let's have a look at the available PMW products:

In [46]:
from gpm_api.io.products import GPM_PMW_2A_GPROF_RS_products
GPM_PMW_2A_GPROF_RS_products()

['2A-GMI',
 '2A-TMI',
 '2A-SSMI-F08',
 '2A-SSMI-F10',
 '2A-SSMI-F11',
 '2A-SSMI-F13',
 '2A-SSMI-F14',
 '2A-SSMI-F15',
 '2A-SSMIS-F16',
 '2A-SSMIS-F17',
 '2A-SSMIS-F18',
 '2A-SSMIS-F19',
 '2A-AMSR2-GCOMW1',
 '2A-AMSRE-AQUA',
 '2A-AMSUB-NOAA15',
 '2A-AMSUB-NOAA16',
 '2A-AMSUB-NOAA17',
 '2A-MHS-METOPA',
 '2A-MHS-METOPB',
 '2A-MHS-METOPC',
 '2A-MHS-NOAA18',
 '2A-MHS-NOAA19',
 '2A-ATMS-NOAA20',
 '2A-ATMS-NPP']

# 1. Data download

Now let's download a 2A PMW product over a couple of hours.

In [ ]:
# Specify the time period you are interested in 
start_time = datetime.datetime.strptime("2020-08-01 12:00:00", "%Y-%m-%d %H:%M:%S")
end_time = datetime.datetime.strptime("2020-08-02 12:00:00", "%Y-%m-%d %H:%M:%S")
# Specify the product and product type 
product = "2A-MHS-METOPB" # "2A-GMI", "2A-SSMIS-F17", ...
product_type = "RS"    
# Specify the version
version = 7

# Download the data
gpm_api.download(
    product=product,
    product_type=product_type,
    version=version,
    start_time=start_time,
    end_time=end_time,
    force_download=False,
    verbose=True,
    progress_bar=True,
    check_integrity=False,
)


Once, the data are downloaded on disk, let's load the 2A product and look at the dataset structure.

## 2. Data Loading

In [ ]:
# Load the dataset
# - If chunks is not None, it does not load the data in RAM memory !
# - If scan_mode is not specified, it automatically load one! 
ds = gpm_api.open_dataset(
    product=product,
    product_type=product_type,
    version=version,
    start_time=start_time,
    end_time=end_time,
    chunks="auto",
)
ds

If you want to load another `scan_mode`, first have a look at the available ones:

In [ ]:
gpm_api.available_scan_modes(product=product, version=version)

and then specify the `scan_mode` argument in `open_dataset`:

In [ ]:
ds = gpm_api.open_dataset(
    product=product,
    product_type=product_type,
    version=version,
    start_time=start_time,
    end_time=end_time,
    scan_mode="S1",
    chunks="auto",
)
ds

You can list variables, coordinates and dimensions with the following methods

In [ ]:
# Available variables
variables = list(ds.data_vars)
print("Available variables: ", variables)
# Available coordinates 
coords = list(ds.coords)
print("Available coordinates: ", coords)
# Available dimensions 
dims = list(ds.dims)
print("Available dimensions: ", dims)

As you see, every variable has a prefix which indicates the group in the original HDF file where the variable is stored. 
You can remove the prefix when opening the dataset by specifying `prefix_group=False`. 
You can also directly load only a subset of variables, by specifying the `variables` argument. 

In [ ]:
# List some variables of interest 
variables = [
   "surfacePrecipitation",
    "rainWaterPath",
    "iceWaterPath",
    "cloudWaterPath"
]
# Load the dataset
ds = gpm_api.open_dataset(
    product=product,
    product_type=product_type,
    version=version,
    start_time=start_time,
    end_time=end_time,
    variables=variables,
    prefix_group=False,
)
ds

To select the DataArray corresponding to a single variable:

In [ ]:
variable = "surfacePrecipitation"  
da = ds[variable]
da

To extract from the DataArray the numerical array you use:

In [ ]:
print("Data type of numerical array: ", type(da.data))
da.data

If the numerical array data type is `dask.Array`, it means that the data are not yet loaded into RAM memory. 
To put the data into memory, you need to call the method `compute`, either on the xarray object or on the numerical array.

In [ ]:
# Option 1 
da_opt1 = da.compute()
print("Data type of numerical array: ", type(da_opt1.data))
da_opt1.data

In [ ]:
# Option 2
print("Data type of numerical array: ", type(da.data.compute()))
da.data.compute()

## 3. Dataset manipulations

Now, let's first have a look at the methods provided by GPM-API

In [ ]:
variable = "surfacePrecipitation"
da = ds[variable]
print("xr.Dataset gpm_api methods:", dir(ds.gpm_api))
print("")
print("xr.DataArray gpm_api methods:", dir(da.gpm_api))


The GPM products are either ORBIT (i.e. PMW and RADAR) or GRID (i.e. IMERG) based.
You can check the support of the data with the methods `is_grid` and `is_orbit`. 

In [ ]:
print("Is GPM ORBIT data?: ", ds.gpm_api.is_orbit)
print("Is GPM GRID data?: ", ds.gpm_api.is_grid)

To check Whether the loaded GPM PMW product has contiguous scans, you can use:

In [ ]:
print(ds.gpm_api.has_contiguous_scans)
print(ds.gpm_api.is_regular)

In case there are non-contiguous scans, you can obtain the along-track slices over which the dataset is regular:

In [ ]:
list_slices = ds.gpm_api.get_slices_contiguous_scans() 
print(list_slices)

You can then select a regular portion of the dataset with:

In [ ]:
slc = list_slices[0]
print(slc)

In [ ]:
ds_regular = ds.isel(along_track=slc)
ds_regular.gpm_api.is_regular

To instead check if the xr.Dataset have just the 2D spatial dimensions, you can use: 

In [ ]:
ds.gpm_api.is_spatial_2d  

## 4. Product visualization

The GPM-API provides two ways of displaying the data:
- The `plot_map` method plot the data in a geographic projection using the Cartopy `pcolormesh` method
- The `plot_image` method plot the data as an image using the Maplotlib `imshow` method

Let's start by plotting the PMW scan in the geographic space

In [ ]:
da = ds[variable].isel(along_track=slice(0,8000))
da.gpm_api.plot_map()     

and now as an image, in "swath" view:

In [ ]:
da.gpm_api.plot_image()

To facilitate the creation of a figure title, GPM-API also provide a `title` method:

In [ ]:
# Title for a single-timestep dataset
print(ds[variable].gpm_api.title(add_timestep=True))
print(ds[variable].gpm_api.title(add_timestep=False))

To instead zoom on a specific regions of a `plot_map` figure, you can use the axis method `set_extent`. 
Note that render the image with this approach can be quite slow, because `plot_map` plots all the data, and then restrict the figure extent over the area of interest. For a more efficient approach, see section `6. Dataset cropping`. 

In [ ]:
from gpm_api.utils.countries import get_country_extent
title = ds.gpm_api.title(add_timestep=False)
extent = get_country_extent("United States")
print("Extent: ", extent)
da = ds[variable].isel(along_track=slice(0, 8000))
p = da.gpm_api.plot_map()  
_ = p.axes.set_extent(extent)
_ = p.axes.set_title(label=title)

You can also customize the geographic projection, by specifying the wished Cartopy projection.
The available projections are [listed here]( https://scitools.org.uk/cartopy/docs/latest/reference/projections.html?highlight=projections)

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from gpm_api.visualization.plot import plot_cartopy_background

# Define some figure options
dpi = 100
figsize = (12, 10)

# Example of polar Cartopy projection
crs_proj = ccrs.Orthographic(180, -90)

# Subset the data for fast rendering
da = ds[variable].isel(along_track=slice(0, 8000))

# Create the map
fig, ax = plt.subplots(subplot_kw={"projection": crs_proj}, figsize=figsize, dpi=dpi)
plot_cartopy_background(ax)
da.gpm_api.plot_map(ax=ax)
ax.set_global()

It is possible to further customize these figures in multiply ways. For example by specifying the own colormap:

In [ ]:
da.gpm_api.plot_map(cmap="Spectral", vmin=0.1, vmax=100)

## 5. Dataset cropping

GPM-API provides methods to easily spatially subset orbits by extent, country or continent.
Note however, that an area can be crossed by multiple orbits. In other words, multiple orbit slices in along-track direction can intersect the area of interest. The method `get_crop_slices_by_extent`, `get_crop_slices_by_country` and ` get_crop_slices_by_continent` enable to retrieve the orbit portions intersecting the area of interest. 

In [ ]:
# Subset the data for fast rendering
da = ds[variable].isel(along_track=slice(0, 8000))

# Crop by extent                                      
extent = (-172, -67, 19, 72) # (xmin, xmax, ymin, ymax)
list_slices = da.gpm_api.get_crop_slices_by_extent(extent)
print(list_slices)
for slc in list_slices:
    da_subset = da.isel(along_track=slc)
    slice_title = da_subset.gpm_api.title(add_timestep=True)
    p = da_subset.gpm_api.plot_map()  
    p.axes.set_extent(extent)
    p.axes.set_title(label=slice_title)


In [ ]:
# Crop by country
# - Option 1
list_slices = da.gpm_api.get_crop_slices_by_country("United States")
print(list_slices)
# - Option 2
from gpm_api.utils.countries import get_country_extent 
extent = get_country_extent("United States")
list_slices = da.gpm_api.get_crop_slices_by_extent(extent)
print(list_slices)
# - Plot the swath crossing the country
for slc in list_slices:
    da_subset = da.isel(along_track=slc)
    slice_title = da_subset.gpm_api.title(add_timestep=True)
    p = da_subset.gpm_api.plot_map()
    p.axes.set_extent(extent)
    p.axes.set_title(label=slice_title)

In [ ]:
# Crop by continent          
# - Option 1
list_slices = da.gpm_api.get_crop_slices_by_continent("South America")
print(list_slices)
# - Option 2
from gpm_api.utils.continents import get_continent_extent 
extent = get_continent_extent("South America")
list_slices = da.gpm_api.get_crop_slices_by_extent(extent)
print(list_slices)
# - Plot the swath crossing the country
for slc in list_slices:
    da_subset = da.isel(along_track=slc)
    slice_title = da_subset.gpm_api.title(add_timestep=True)
    p = da_subset.gpm_api.plot_map() 
    p.axes.set_extent(extent)
    p.axes.set_title(label=slice_title)
